In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('/content/fraud_detection_dataset_10k (1).csv')
df.head(5)

,Message,Is_Fraudulent
0,is PM. doctor's Your for Reminder: appointment...,No
1,vacation! card information. your a credit prov...,Yes
2,"$1,000,000. Congratulations! You won",Yes
3,your password. reset compromised. here has to ...,Yes
4,account. Confirm your We've identity. activity...,Yes


In [3]:
def tokenized(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'",'')
  text = text.split()
  return text

In [4]:
tokenized("What is your Name?")

['what', 'is', 'your', 'name']

In [5]:
vocab = {'<UNK>': 0}

In [6]:
def build_vocab(row):
  print(row['Message'] , row['Is_Fraudulent'])
  tokenized_question = tokenized(row['Message'])
  tokenized_answer = tokenized(row['Is_Fraudulent'])
  tokenized_merge = tokenized_question + tokenized_answer
  for word in tokenized_merge:
    if word not in vocab:
      vocab[word] = len(vocab)

In [7]:
df.apply(build_vocab, axis=1)

is PM. doctor's Your for Reminder: appointment scheduled 3! No
vacation! card information. your a credit provide free Win Just Yes
$1,000,000. Congratulations! You won Yes
your password. reset compromised. here has to been Your Click account? Yes
account. Confirm your We've identity. activity your detected unusual in! Yes
in your unusual We've detected activity identity. account. Confirm your Yes
Congratulations! won $1,000,000. You? Yes
Please send updates. me the project: No
detected account. your activity unusual your Confirm in We've identity. Yes
the you conference at See tomorrow.? No
will account Final locked. be warning: Your PayPal Yes
has security now. social Act compromised. been Your number Yes
here been has reset to Your password. Click compromised. account your Yes
clicking a Exclusive link. offer! by this free Get iPhone Yes
you the See at conference tomorrow. No
by Can EOD? the report review you No
Your Reminder: is doctor's PM. 3 for scheduled appointment; No
project t

,0
0,None
1,None
2,None
3,None
4,None
...,...
4995,None
4996,None
4997,None
4998,None


In [8]:
vocab

{'<UNK>': 0,
 'is': 1,
 'pm.': 2,
 'doctors': 3,
 'your': 4,
 'for': 5,
 'reminder:': 6,
 'appointment': 7,
 'scheduled': 8,
 '3!': 9,
 'no': 10,
 'vacation!': 11,
 'card': 12,
 'information.': 13,
 'a': 14,
 'credit': 15,
 'provide': 16,
 'free': 17,
 'win': 18,
 'just': 19,
 'yes': 20,
 '$1,000,000.': 21,
 'congratulations!': 22,
 'you': 23,
 'won': 24,
 'password.': 25,
 'reset': 26,
 'compromised.': 27,
 'here': 28,
 'has': 29,
 'to': 30,
 'been': 31,
 'click': 32,
 'account': 33,
 'account.': 34,
 'confirm': 35,
 'weve': 36,
 'identity.': 37,
 'activity': 38,
 'detected': 39,
 'unusual': 40,
 'in!': 41,
 'in': 42,
 'please': 43,
 'send': 44,
 'updates.': 45,
 'me': 46,
 'the': 47,
 'project:': 48,
 'conference': 49,
 'at': 50,
 'see': 51,
 'tomorrow.': 52,
 'will': 53,
 'final': 54,
 'locked.': 55,
 'be': 56,
 'warning:': 57,
 'paypal': 58,
 'security': 59,
 'now.': 60,
 'social': 61,
 'act': 62,
 'number': 63,
 'clicking': 64,
 'exclusive': 65,
 'link.': 66,
 'offer!': 67,
 'by':

In [9]:
def text_indice(text,vocab):
  indice = []
  for text in tokenized(text):
    if text in vocab:
      indice.append(vocab[text])
    else:
      indice.append(vocab['<UNK>'])
  return indice

In [10]:
text_indice("what is Nishant",vocab)

[0, 1, 0]

In [11]:
import torch
from torch.utils.data import Dataset,DataLoader

In [12]:
class QA(Dataset):
  def __init__(self,df,vocab):
    self.df = df
    self.vocab = vocab
  def __len__(self):
    return len(self.df)
  def __getitem__(self, index):
    # Convert 'Is_Fraudulent' column to string before tokenization
    nuemerical_q =text_indice(self.df.iloc[index]['Message'],self.vocab)
    nuemerical_a = text_indice(str(self.df.iloc[index]['Is_Fraudulent']),self.vocab) # Convert to string

    return torch.tensor(nuemerical_q) , torch.tensor(nuemerical_a)

In [13]:
dataset = QA(df,vocab)
dataloader = DataLoader(dataset,batch_size=1,shuffle=True)

In [14]:
for q , a in dataloader:
  print(q , a )

tensor([[ 14, 113, 114,  23, 115, 112, 116]]) tensor([[10]])
tensor([[ 39,  36,   4,  38,  37,   4,  34,  40,  42, 299]]) tensor([[20]])
tensor([[60, 63, 29,  4, 27, 62, 31, 59, 61]]) tensor([[20]])
tensor([[102,  56, 103,  99, 100,  50, 101]]) tensor([[10]])
tensor([[100,  99, 102, 103,  50,  56, 101]]) tensor([[10]])
tensor([[74, 23, 47, 68, 73, 72, 75]]) tensor([[10]])
tensor([[ 3,  1, 76,  8,  6,  2,  4,  7,  5]]) tensor([[10]])
tensor([[ 22,  24,  21, 159]]) tensor([[20]])
tensor([[ 83,  80,  84,  81,   4,  79, 486]]) tensor([[20]])
tensor([[56, 57, 58, 33, 55, 53, 54,  4]]) tensor([[20]])
tensor([[44, 47, 43, 46, 78, 45]]) tensor([[10]])
tensor([[52, 49, 51, 47, 50, 23]]) tensor([[10]])
tensor([[78, 47, 46, 43, 44, 45]]) tensor([[10]])
tensor([[  1, 132, 131,   4,   4,   5, 130, 146]]) tensor([[10]])
tensor([[14, 12, 16, 17, 18, 15, 11, 19, 13,  4]]) tensor([[20]])
tensor([[50, 47, 23, 52, 49, 51]]) tensor([[10]])
tensor([[14, 12, 18,  4, 13, 17, 11, 16, 15, 19]]) tensor([[20]])


In [15]:
from torch import nn

In [16]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [17]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)
size_of_tensor = dataset[0][0].size()[0]
a = dataset[0][0].reshape(1,size_of_tensor)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 9])
shape of b: torch.Size([1, 9, 50])
shape of c: torch.Size([1, 9, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [18]:
model = SimpleRNN(len(vocab))


In [19]:
learning_rate = 0.001
epochs = 20

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [21]:
# training loop

for epoch in range(epochs):

  total_loss = 0

  for question, answer in dataloader:

    optimizer.zero_grad()

    # forward pass
    output = model(question)

    # loss -> output shape (1,324) - (1)
    loss = criterion(output, answer[0])

    # gradients
    loss.backward()

    # update
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 735.797551
Epoch: 2, Loss: 8.768008
Epoch: 3, Loss: 0.071265
Epoch: 4, Loss: 0.005816
Epoch: 5, Loss: 0.000403
Epoch: 6, Loss: 0.000013
Epoch: 7, Loss: 21.920318
Epoch: 8, Loss: 4.090080
Epoch: 9, Loss: 0.003278
Epoch: 10, Loss: 0.001047
Epoch: 11, Loss: 0.000092
Epoch: 12, Loss: 0.000011
Epoch: 13, Loss: 0.000000
Epoch: 14, Loss: 0.000000
Epoch: 15, Loss: 0.000000
Epoch: 16, Loss: 0.000001
Epoch: 17, Loss: 31.042740
Epoch: 18, Loss: 0.006448
Epoch: 19, Loss: 0.001477
Epoch: 20, Loss: 0.000187


In [22]:
torch.save(model, 'fraud_detection_model.pth')

In [25]:
# Inference function
def predict(model, message):
    model.eval()
    with torch.no_grad():
        question_tensor = torch.tensor(text_indice(message, vocab)).unsqueeze(0)
        output = model(question_tensor)
        probs = torch.nn.functional.softmax(output, dim=1)
        prediction = 'Yes' if torch.argmax(probs) == 1 else 'No'
        return prediction

# Load model for inference
model.load_state_dict(torch.load('fraud_detection_model.pth'))

# Example prediction
print(predict(model, "You've won a prize $100000"))

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL __main__.SimpleRNN was not an allowed global by default. Please use `torch.serialization.add_safe_globals([SimpleRNN])` or the `torch.serialization.safe_globals([SimpleRNN])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by default with weights_o

In [30]:
def predict(model, question, threshold=0.4):

  # convert question to numbers
  numerical_question = text_indice(question, vocab)

  # tensor
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)

  # send to model
  output = model(question_tensor)

  # convert logits to probs
  probs = torch.nn.functional.softmax(output, dim=1)

  # find index of max prob
  value, index = torch.max(probs, dim=1)

  if value < threshold:
    print("No")

  print(list(vocab.keys())[index])

In [38]:
predict(model, "won $100000")

yes
